##### 
# DenseNet201 model, handling 28 x 28 shape data

In [ ]:
import numpy as np
import pandas as pd
import pylab as pl
from PIL import Image
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import os

import tensorflow as tf
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
train0 = pd.read_csv('../input/digit-recognizer/train.csv')
test0 = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
print(train0.shape)
train0.head()

In [ ]:
print(test0.shape)
test0.head()

In [ ]:
X0 = train0.iloc[:,1:]
y0 = train0.iloc[:,0]

In [ ]:
X1=np.array(X0)
test1=np.array(test0)

print(X1.shape)
print(test1.shape)

In [ ]:
X2=[]
for i in tqdm(range(len(X1))):
    obj0 = X1[i]
    obj1 = Image.fromarray(np.uint8(obj0))
    obj2 = np.asarray(obj1.resize((56,56*3))) 
    X2 += [obj2]

In [ ]:
test1=np.array(test0)
    
test2=[]
for i in tqdm(range(len(test1))):
    obj0 = test1[i]
    obj1 = Image.fromarray(np.uint8(obj0))
    obj2 = np.asarray(obj1.resize((56,56*3))) 
    test2 += [obj2]  

In [ ]:
print(np.array(X2).shape)
print(np.array(test2).shape)

In [ ]:
X=(np.array(X2)).reshape(-1,56*56*3)
test=(np.array(test2)).reshape(-1,56*56*3)

print(X.shape)
print(test.shape)

In [ ]:
binencoder = LabelBinarizer()
y = binencoder.fit_transform(y0)
y

In [ ]:
X_images = X.reshape(-1,56,56,3)
test_images = test.reshape(-1,56,56,3)

print(X_images.shape)
print(test_images.shape)

In [ ]:
plt.figure(figsize=(6,6), dpi=50)
plt.imshow(X1[5].reshape(28,28))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure(figsize=(18,6), dpi=50)
plt.imshow(X_images[5].reshape(56,56*3))
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size = 0.2, random_state=90)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = X_train/255
X_test = X_test/255

X_train = X_train.reshape(-1,56,56,3).astype('float32')
X_test = X_test.reshape(-1,56,56,3).astype('float32')

In [ ]:
test_images2 = test_images/255
test = test_images2.reshape(-1,56,56,3).astype('float32')

In [ ]:
pretrained_model = tf.keras.applications.DenseNet201(input_shape=(56,56,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model.summary()

In [ ]:
his = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=92, verbose=2)

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
pred2=model.predict(test)
print(test.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]

In [ ]:
sample=pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sample

In [ ]:
submit=sample
submit['Label']=PRED
submit.to_csv('submission.csv',index=False)
submit